# prologue

### set up notebook and load package

In [1]:
# load what we need
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp

# data management

### datasets

Several datasets are available as pre-prepared containers that hold the data and some meta-data that is used in the algorithm
Any dataset can be turned into a container by invoking the constructor found in the file structures.py

In [2]:
# datasets might be down-sampled to make them easier to work with
# the full sets are available too
# this is a list of constructors that will be used in the benchmarking

# demo datasets that ship with package. all from UCI unless stated otherwise
# ds.adult_data, ds.adult_samp_data, ds.adult_small_samp_data Large dataset ships with manageable sub samples
# ds.bankmark_data, ds.bankmark_samp_data
# ds.car_data
# ds.cardio_data this is the cardiotocography dataset
# ds.credit_data
# ds.german_data
# ds.lending_data, ds.lending_samp_data, ds.lending_small_samp_data, ds.lending_tiny_samp_data from Kaggle. see datasets_from_source file for links
# ds.nursery_data, ds.nursery_samp_data
# ds.rcdv_data, ds.rcdv_samp_data from US government see datasets_from_source file for links

if False:
    rp.datasets

In [3]:
# example of one dataset
# note: random_state propagates through other functions and is easily updated to allow alternative runs
if False:
    ds.cardio(random_state=123, project_dir=project_dir)

### standardising train-test splitting
Some methods are not available in Python. We want to maintain the same dataset splits no matter which platform. So, the train test data is split with the one-time random seed and the splits are saved to csv in the project folders.

In [4]:
# writes external files
if False:
    rp.export_data_splits(datasets=rp.datasets, project_dir=project_dir, random_state_splits=random_state_splits)

# Experimental Runs
Loop through datasets, actioning the functions in the package to execute a round of experiments and test evaluations.

## 0. Optional Memory and Computation Cost Management
CHIRPS is time economical but memory intensive to compute for lots of instances at once.


### Parallel processing
Scikit takes care of parallel for the RF construction.
We can parallelise the following:
1. the walk of instances down each tree to collect the paths. The paths for many instances are returned in a single array. This parallelises across trees.
2. building CHIRPS and the final explanation (rule). This is a search optimisation and we can parallelise each instance.

This is expecially effective when running batches. For single instances, set both to false to avoid spinning up the parallel infrastructure.

### Preparing unseen data

Again note:
test set has never been "seen" by random forest during training
test set has been only used to assess model (random forest) accuracy - no additional tuning after this
test set has not be involved in generating the explainer

#### Batching
The memory space requirements for all the paths can be reduced by dividing the test set into batches. However this does take longer as there is an overhead to instantiate all the required objects, especially if coupled with parallel processing.
Best compromise could be a small number of larger batches.

### 1. Data and Forest prep
Use the random state splits to do a one-off data split.
Fit the RF to training data, using the iterating random state.
Save the performance metrics on the test set for later review.

### 2. Prepare Unseen Data and Predictions
Important to note:
Test set never "seen" by RF during training.
test set not involved in generating the explainer.
Test set used to evaluate model (random forest) accuracy beyond OOBE scores - no additional tuning based on these results.
Test set used to evaluate explanation scores by leave-one-out method removing the specific instance we're explaining.

Important to note:
We will explain predictions directly from the trained RF. Explanation system makes no compromise on model accuracy.

### 3. CHIRPS algorithm
1. Extract tree prediction paths
2. Freqent pattern mining of paths
3. Score and sort mined path segments
4. Merge path segments into one rule

#### CHIRPS 1
Fit a forest_walker object to the dataset and decision forest. This is a wrapper that will extract the paths of all the given instances. Its main method delivers the instance paths for the remaining steps of the algorithm as a new object: a batch_paths_container. It can also report interesting statistics (treating the forest as a set of random tree-structured variables).

#### CHIRPS 2-4
A batch_CHIRPS_container is fitted with the batch_paths_container returned by the forest walker, and with a sample of data. For CHIRPS, we prefer a large sample. The whole training set or other representative sample will do. This is a wrapper object will execute steps 2-4 on all each the instance-paths in the batch_paths_container.

Important to note:
true_divide warnings are OK! It just means that a continuous variable is unbounded on one side i.e. no greater/less than inequality is used in the specific CHIRPS explanation.

Important note: 
Here we are using the training set to create the explainers. We could use a different dataset as long as it is representative of the training set that built the decision forest. Most important that we don't use the dataset that we wish to explain, so never use the test set, for example.

### 4. Evaluating CHIRPS Explanations
Test set has been used to create an explainer *one instance at a time* and the rest of test set was not "seen" during this construction. To score each explainer, we use test set, leaving out the individual instance being explained. The data_split_container (tt) has a convenience funtion for doing this. All the results are saved to csv files in the project directory.

In [5]:
# CHIRPS default set up
merging_bootstraps = 20
pruning_bootstraps = 20
delta = 0.1

forest_walk_async=True
chirps_explanation_async=False

n_instances = 10000

# model = 'RandomForest'
# model = 'AdaBoost1'
model = 'AdaBoost2'
# model = 'GBM'

do_Anchors = True
do_dfrgTrs = True

datasets = [rp.datasets[0]] # here can opt for just one, e.g. [rp.datasets[0]] (as an iterator)
start_instance = 0 # here can opt to start at a specific instance

project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

tuning = {'grid' : None, 'override' : False}
if model == 'RandomForest':
    tuning.update({'grid' : None}) # defaults to n_trees [200, 400, ..., 1600]
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance, verbose=verbose)

    kwargs = {'support_paths' : 0.1, 'alpha_paths' : 0.5, 'disc_path_bins' : 4,
             'score_func' : 1, 'weighting' : 'chisq',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose, **control)
    
    if do_Anchors:
        control = {'method' : 'Anchors',
                   'model' : model,
                   'n_instances' : n_instances,
                   'random_state' : random_state_exp}
        rp.do_benchmarking(benchmark_items, verbose, **control)
        
    if do_dfrgTrs:
        control = {'method' : 'defragTrees',
                    'Kmax' : 1, 'restart' : 1, 'maxitr' : 1,
                    'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp}
        rp.do_benchmarking(benchmark_items, verbose, **control)
            
elif model == "AdaBoost1":
    algo = 'SAMME'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)
    
    kwargs = {'paths_lengths_threshold' : 5,
             'support_paths' : 0.1, 'alpha_paths' : 0.0,
             'disc_path_bins' : 4, 'disc_path_eqcounts' : True,
             'score_func' : 1, 'weighting' : 'chisq',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose=True, **control)
    
elif model == 'AdaBoost2':
    algo = 'SAMME.R'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)
    
    kwargs = {'paths_lengths_threshold' : 5,
                 'support_paths' : 0.01, 'alpha_paths' : 0.0,
                 'disc_path_bins' : 4, 'disc_path_eqcounts' : True,
                 'score_func' : 1, 'weighting' : 'chisq',
                 'merging_bootstraps' : merging_bootstraps,
                 'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
    
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose=True, **control)
    
else: # GBM
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)

Preprocessing adult_small_samp data and model for adult_small_samp with random state = 123
Split data into main train-test and build forest
using previous tuning parameters


C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Best OOB Accuracy Estimate during tuning: 0.8473
Best parameters:{'algorithm': 'SAMME.R', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 200, 'random_state': 123}

Discretise data and train model for Anchors
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.8473
Best parameters:{'algorithm': 'SAMME.R', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None

C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1442: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1447: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1447: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1442: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin


pruned away: restoring previous rule


...

Working on CHIRPS for instance 725 of 733


C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1447: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1442: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1447: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1442: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1447: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lower

Working on CHIRPS for instance 730 of 733


C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1442: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1447: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1447: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1442: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1447: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lower

CHIRPS time elapsed: 1922.8156 seconds
CHIRPS with async = False

Evaluating found explanations
Results saved to V:\whiteboxing\tests\adult_small_samp\AdaBoost2_CHIRPS_rnst_123
CHIRPS batch results eval time elapsed: 23.6349 seconds



In [6]:
stop

NameError: name 'stop' is not defined

## Sensitivity Analysis

In [ ]:
# load what we need
import numpy as np
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp

# CHIRPS default set up
merging_bootstraps = 20
pruning_bootstraps = 20
delta = 0.1 # prune rule terms if loss of precision no greater than delta

forest_walk_async=True
chirps_explanation_async=True

n_instances = 10000

# model = 'RandomForest'
model = 'AdaBoost1'
# model = 'AdaBoost2'
# model = 'GBM'

do_Anchors = False
do_dfrgTrs = False

datasets = rp.datasets # here can opt for just one, e.g. [rp.datasets[0]] (as an iterator)
start_instance = 0 # here can opt to start at a specific instance

project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

tuning = {'grid' : None, 'override' : False}

# for troubleshooting, can isolate an instance and do like
# bi_copy['cardio']['main']['ds_container'].current_row_test = 110

if model == 'RandomForest':
    tuning.update({'grid' : None}) # defaults to n_trees [200, 400, ..., 1600]
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance, verbose=verbose)
    
    alpha_paths = np.tile([0.9, 0.5, 0.1], 24)
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 12)
    score_func = np.tile(np.repeat([5, 3, 1], 6), 4)
    support_paths = np.tile(np.repeat([0.1, 0.05], 18), 2)
    weighting = np.repeat(['chisq', 'nothing'], 36)

    kwargs_grid = {k : {'alpha_paths' : ap, 'disc_path_bins' : dpb, 'score_func' : sf, 'weighting' : w, 'support_paths' : sp,
                       'merging_bootstraps' : merging_bootstraps, 'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta} 
        for k, ap, dpb, sf, w, sp in zip(range(72), alpha_paths, disc_path_bins, score_func, weighting, support_paths)}
    
    
    for kwargs in kwargs_grid:
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'save_sensitivity_path' : 'rf_sensitivity'}


        rp.do_benchmarking(bi_copy, verbose, **control)
        
elif model in ('AdaBoost1', 'AdaBoost2'):
    if model == 'AdaBoost1':
        algo = 'SAMME'
        save_sensitivity_path = 'ada1_sensitivity'
    else:
        algo = 'SAMME.R'
        save_sensitivity_path = 'ada2_sensitivity'
        
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)
    
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 8)
    support_paths = np.tile([0.05, 0.02, 0.01], 12)
    weighting = np.repeat(['chisq', 'kldiv', 'lodds', 'nothing'], 6)
    
    kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                        'disc_path_bins' : dpb, 'disc_path_eqcounts' : True,
                        'score_func' : 1, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'delta' : delta} 
    for k, dpb, w, sp in zip(range(24), disc_path_bins, weighting, support_paths)}
    
    for kwargs in kwargs_grid:
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'save_sensitivity_path' : save_sensitivity_path}
 
        rp.do_benchmarking(bi_copy, verbose, **control)